--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Install relevant packages and modules

In [3]:
!apt-get install -y dcm2niix # '!' runs commands in the console
!apt-get install -y parallel

# dependencies for freesurfer
!apt-get install -y wget
!apt-get install -y grep
!apt-get install -y tcsh
!apt-get install -y bc

!pip install nibabel

import os
import glob
import nibabel as nib
import numpy as np

# install freesurfer (might take a few minutes)

# if you are having issues downloading freesurfer, it is likely due to the version selected here
!wget -O freesurfer.tar.gz https://freesurfer.net/pub/dist/freesurfer/7.4.1/freesurfer-linux-ubuntu22_amd64-7.4.1.tar.gz
!tar -xzf freesurfer.tar.gz

# set the relevant freesurfer directories
os.environ['FREESURFER_HOME'] = '/content/freesurfer'
os.environ['SUBJECTS_DIR'] = '/content/freesurfer_output'
os.environ['PATH'] += ':/content/freesurfer/bin'

!source /content/freesurfer/SetUpFreeSurfer.sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
dcm2niix is already the newest version (1.0.20211006-1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
parallel is already the newest version (20210822+ds-2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
grep is already the newest version (3.7-1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tcsh is already the newest version

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Add the FreeSurfer License path. This is free to get but due to their policy I cannot include it here. It is available at https://surfer.nmr.mgh.harvard.edu/registration.html

In [ ]:
from google.colab import files
license = files.upload()

# change this to what you named the license
your_file_name = 'freesurfer_license.txt'

# set the enviorment to use the license
os.environ['FS_LICENSE'] = f'/content/{your_file_name}'

Saving freesurfer_license.txt to freesurfer_license.txt


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Get anonymized (T1 weighted) dicom files for analysis, and create temporary folders to store these and other files.

In [ ]:
# clone the repository to get the raw dicoms
!git clone https://github.com/datalad/example-dicom-structural

# create folders for the NIfTI conversions and corresponding recons
!mkdir /content/nii_files/
!mkdir /content/freesurfer_output/

'''
Note: "T1 weighting" refers to a form of MRI image processing performed by technicians.
Online resources will nearly always provide processed MRI images,
though they may not be T1 weighted
''';

Cloning into 'example-dicom-structural'...
remote: Enumerating objects: 393, done.
remote: Total 393 (delta 0), reused 0 (delta 0), pack-reused 393 (from 1)
Receiving objects: 100% (393/393), 15.45 MiB | 14.97 MiB/s, done.
Resolving deltas: 100% (223/223), done.


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Convert the DICOMs to NIfTIs

In [ ]:
# prepare paths
INPUT_PATH = '/content/example-dicom-structural/dicoms/'
OUTPUT_PATH = '/content/nii_files/'

# grep to ignore some warnings regarding the manufacturer (since we're using sample dicoms)
!dcm2niix -o '{OUTPUT_PATH}' '{INPUT_PATH}' | grep -v "Unknown manufacturer"

Chris Rorden's dcm2niiX version v1.0.20211006  (JP2:OpenJPEG) GCC11.2.0 x86-64 (64-bit Linux)
Found 384 DICOM file(s)
Convert 384 DICOM as /content/nii_files/dicoms_anat-T1w_20130717141500_401 (274x384x384x1)
Conversion required 0.412129 seconds (0.389445 for core code).


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Reconstruct (recon) the NIfTI files


Parallel allows you to recon multiple subjects simultaneously by recruiting n CPU cores (determined by --jobs n). It is irrelevant here, but is good to be aware of

Recons take a long time (4 hours+ per nii) and are quite large (~100Mb). For this reason, I have included the finished recon within this repository, with only the essential files kept

In [ ]:
# prepare paths
INPUT_PATH = '/content/freesurfer_output'
nii_paths = glob.glob('/content/nii_files/*.nii') # list of paths
ALL_NII = ' '.join(nii_paths)  # format that parallel wants

# use parallel to execute recon-all on each NIfTI file
!parallel --jobs 1 recon-all -i {} -s {/.} -all ::: /content/nii_files/*.nii

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Load in the sample recon

In [11]:
!git clone https://github.com/SamAndTheSun/sMRI_BrainAge_Tutorial.git
recon_path = '/content/sMRI_BrainAge_Tutorial/sample_recon'

fatal: destination path 'sMRI_BrainAge_Tutorial' already exists and is not an empty directory.


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

Deep Learning Analysis Example 1: Using the brain.mgz (CNN)


The brain.mgz file represents the combination of each individual "slice" of the brain stitched together to form a single cohesive volume. We can use this file to construct a CNN, trained on 3D images, to predict brain age

In [ ]:
# load in the brain.mgz files
brain_files = glob.glob(f'{recon_path}/*/mri/brain.mgz')

Convert the brain files to the same space, so that the model is fed consistent data

In [ ]:
# get the converted files

for subj_brain in brain_files:

  # convert the brain to
  mri_vol2vol -i {} -s {/.} -all

  # get the brain data
  mgz_file = nib.load(subj_brain)
  brain_data = mgz_file.get_fdata()

  # convert the brain data to native space using freesurfer
  mri_vol2vol -i {} -s {/.} -all


Deep Learning Analysis Example 2: Using the surf files (graphSAGE)

The .pial files (one for each hemisphere) represent the geometric vertices and faces (i.e. the connections between vertices) for each individual's cortex. Freesurfer also provides other files, such as the cortical thickness (.thickness) and white-grey matter intensity ratio (.w-g.pct.mgh) which provide information on each of these vertices.

We can use these files in conjunction to construct a graph of each individual's brain, with vertices serving as nodes, faces serving as edges, and attributes serving as features.

In [49]:
# get each subject (only one here)
subj_files = glob.glob(f'{recon_path}')

# load in the pial files
pial_files = glob.glob(f'{recon_path}/surf/*h.pial') # *h because we want both hemispheres

# load in the thickness files
thickness_files = glob.glob(f'{recon_path}/surf/*h.thickness')

# load in the white-grey matter intensity ratio files
wg_ratio_files = glob.glob(f'{recon_path}/surf/*h.w-g.pct.mgh')

# glob uses a random sort order, so we sort alphabetically to match everything
subj_files = sorted(subj_files)
pial_files = sorted(pial_files)
thickness_files = sorted(thickness_files)
wg_ratio_files = sorted(wg_ratio_files)

# construct a dictionary with structure: subjects -> nodes -> features/edges
training_data = {}
for i, subj in enumerate(subj_files):

  # get the last part of the path
  subj_id = subj.split('/')[-1]

  # get the vertice and face data for the subject
  lh_vertices, lh_faces = nib.freesurfer.read_geometry(pial_files[i]) # we know lh is before rh because we sorted alphabetically
  rh_vertices, rh_faces = nib.freesurfer.read_geometry(pial_files[i+1])

  # combine them into a single array
  vertices = np.vstack((lh_vertices, rh_vertices+(np.max(lh_vertices)+1))) # vertices uses relative node index (min=0), so we need to account for this

  # do the same for every other file type
  lh_thickness = nib.freesurfer.io.read_morph_data(thickness_files[i]) # be mindful of which nib reading varient to use
  rh_thickness = nib.freesurfer.io.read_morph_data(thickness_files[i+1])
  thickness = np.hstack((lh_thickness, rh_thickness)) # not relative, notice the use of h-stack for single-dimension variables
  #
  lh_ratio = nib.load(wg_ratio_files[i]).get_fdata()
  rh_ratio = nib.load(wg_ratio_files[i+1]).get_fdata()
  ratio = np.vstack((lh_ratio, rh_ratio)) # not relative
  ratio = ratio.squeeze() # this has dimensions (n_nodes, 1, 1) otherwise

  # create a node for each vertice and a seperate list for the edges
  training_data[subj_id] = [[] for _ in range(vertices.shape[1])]

  # for each vertice add the corresponding features
  for n, node in enumerate(training_data[subj_id]):
    print(node)
    node.append(vertices[:, n])
    node.append(thickness[n])
    node.append(ratio[n])

  print(training_data)

  # add an additional key representing the edges




[]


NameError: name 'waaargh' is not defined